In [5]:
# Cell 1: Import Libraries
import pandas as pd
import os
import numpy as np
from keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Embedding, LSTM, Dropout, BatchNormalization, Bidirectional
from keras.optimizers import Adam
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM, Dropout
from tensorflow.keras.layers import Input
from tensorflow.keras import layers, models


def create_cnn_model(input_shape, num_classes):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))  # Dropout after first max pooling

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))  # Dropout after second max pooling

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))  # Dropout before the final output layer

    model.add(Dense(num_classes, activation='softmax'))
    
    return model

def create_lstm_model(input_shape, num_classes):
        
    model = Sequential()
    model.add(Embedding(input_dim=10000, output_dim=128, input_length=input_shape))  # Adjust parameters as needed
    model.add(LSTM(128, return_sequences=True))
    model.add(Dropout(0.25))  # Add dropout layer after LSTM
    model.add(LSTM(64))  # You can adjust this layer as necessary
    model.add(Dropout(0.5))  # Add another dropout layer
    model.add(Dense(num_classes, activation='softmax'))
    return model

    # Combined Model
def create_combined_model(image_input_shape, recipe_input_shape, num_classes):
    # Image input
    image_input = Input(shape=image_input_shape)
    
    # CNN model for extracting features from food image
    cnn_model = create_cnn_model(image_input_shape, num_classes)
    cnn_output = cnn_model(image_input)
    cnn_model.summary()
    # Recipe input (textual)
    recipe_input = Input(shape=(recipe_input_shape,))
    
    # LSTM model for generating recipe (textual data)
    lstm_model = create_lstm_model(recipe_input_shape, num_classes)
    lstm_output = lstm_model(recipe_input)
    lstm_model.summary()
    # Combine CNN and LSTM outputs (e.g., concatenate or other methods)
    combined_output = layers.concatenate([cnn_output, lstm_output], axis=-1)
    
    # Final output layer
    final_output = Dense(num_classes, activation='softmax')(combined_output)

    # Create the final model
    model = models.Model(inputs=[image_input, recipe_input], outputs=final_output)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Example input shapes
image_input_shape = (224, 224, 3)  # Example: 224x224 RGB images
recipe_input_shape = 50  # Example: 50 words in a recipe sequence
num_classes = 9  # Example: Number of food classes or recipe types

# Create the combined model
model = create_combined_model(image_input_shape, recipe_input_shape, num_classes)

# Display the model summary
model.summary()

c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 186624)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    23,888,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 9)              │         1,161 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,908,553 (91.20 MB)

 Trainable params: 23,908,553 (91.20 MB)

...

Model: "functional_11"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_2       │ (None, 50)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential          │ (None, 9)         │ 23,908,553 │ input_layer[0][0] │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_1        │ (None, 9)         │  1,461,577 │ input_layer_2[0]… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 18)        │          0 │ sequential[0][0], │
│ (Concatenate)       │                   │            │ sequential_1[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 9)         │        171 │ concatenate[0][0] │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 25,370,301 (96.78 MB)

 Trainable params: 25,370,301 (96.78 MB)

 Non-trainable params: 0 (0.00 B)